# Introducción a las técnicas Zero-shot, Few-shot y Chain of Thought

En este cuaderno aprenderás sobre tres técnicas fundamentales usadas con los grandes modelos de lenguaje (LLMs):

1. **Zero-shot learning**: Resolver tareas sin recibir ningún ejemplo.
2. **Few-shot learning**: Resolver tareas con un número reducido de ejemplos.
3. **Chain of Thought (Cadena de razonamiento)**: Generar razonamientos intermedios paso a paso para mejorar la precisión del modelo.

Utilizaremos el modelo Falcon 7B en su versión instruida.

¿Que significa eso de que se la versión instruida? Quiere decir que el modelo no solo ha sido entrenado con textos para que aprenda un vocabulario y lenguas, sino que ademas, ha sido entrenado con una serie de instrucciones para que sea capaz de realizar ciertas tareas del ámbito del proesamiento de lenguaje natural, como pueda ser la traducción, la generación de texto, la clasificación de texto, etc.

## Paso 1: Configuración del entorno

Para empezar, instalaremos las bibliotecas necesarias y nos aseguraremos de tener acceso a una GPU. En caso de no tener acceso, recuerda habilitar un entorno con GPU en Google Colab.

In [1]:
# Comprueba si estás utilizando una GPU
import torch
if torch.cuda.is_available():
    print("¡GPU detectada! Todo listo para usar.")
else:
    print("No se detectó GPU. Ve a Entorno de ejecución > Cambiar tipo de entorno de ejecución > GPU")

¡GPU detectada! Todo listo para usar.


Instalamos la última versión de bitsandbytes, librería que usaremos mas tarde

In [2]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

## Paso 2: Importar el modelo y preparar la interfaz

Utilizaremos el modelo Falcon 7B instruido, obtenido del repositorio de modelos de Hugging Face. Puedes consultar toda la información acerca de este modelo en [este enlace](https://huggingface.co/tiiuae/falcon-7b-instruct).

Lo cargaremos cuantizado, es decir, reduciremos el tamaño de los datos que contiene de 16 bits a 8 bits. De no reducir el tamaño, el modelo no cabría en Google Colab.

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Configurar la cuantización de 8 bits
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16
)

MODEL_NAME = "tiiuae/falcon-7b-instruct"

# Cargar el tokenizador
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Cargar el modelo cuantizado
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=quantization_config,
    device_map="auto"
)

Unused kwargs: ['bnb_8bit_compute_dtype']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [4]:
from transformers import pipeline

#Importamos una tubería predefinida para la generación de texto, en la que especificamos el modelo, el tokenizador y la GPU a utilizar
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

print(f"Modelo {MODEL_NAME} cargado correctamente.")

Device set to use cuda:0


Modelo tiiuae/falcon-7b-instruct cargado correctamente.


## Paso 3: Zero-shot Learning

La técnicas de Zero-shot learning permite a los modelos responder a preguntas sin recibir ningún ejemplo adicional o contexto. Es decir, confía completamente en su conocimiento preentrenado. Por tanto, y dada esa confianza depositada en el modelo, es importante que el modelo haya sido entrenado con una gran cantidad de datos y que haya sido instruido para realizar tareas de procesamiento de lenguaje natural. En caso contrario, podríamos obtener respuestas incorrectas.

Probaremos a preguntarle al modelo cuál es la capital de España, sin darle ningún contexto adicional.

In [5]:
# Pregunta simple (Zero-shot learning)
pregunta = "¿Cuál es la capital de España?"

respuesta = generator(pregunta, max_length=50, num_return_sequences=1, temperature=0.0, early_stopping=True, num_beams=2)
print("Pregunta:", pregunta)
print("Respuesta del modelo:", respuesta[0]['generated_text'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Pregunta: ¿Cuál es la capital de España?
Respuesta del modelo: ¿Cuál es la capital de España?
The capital of Spain is Madrid.


Como podemos observar con la respuesta, el modelo ha sido capaz de responder correctamente a la pregunta sin necesidad de recibir ningún ejemplo adicional. Por contra, vemos que la respuesta que nos ha otorgado está en inglés, pese a que le hemos preguntado en castellano.

Además, la respuesta que da la realiza en una linea separada. ¿Que pasará si le damos un contexto adicional?

## Paso 4: Few-shot Learning

En este caso, y continuado el ejemplo presentado en el apartado anterior, preguntaremos al modelo por cuál es la capital de España, pero en este caso, proporcionamos algunos ejemplos contextuales que guían al modelo para responder de manera más precisa.

In [6]:
# Few-shot learning: Proporcionamos algunos ejemplos
ejemplos = (
    "Pregunta: ¿Cuál es la capital de Francia? Respuesta: París.\n"
    "Pregunta: ¿Cuál es la capital de Alemania? Respuesta: Berlín.\n"
    "Pregunta: ¿Cuál es la capital de Italia? Respuesta: Roma.\n"
    "Pregunta: ¿Cuál es la capital de España? Respuesta:"
)
respuesta = generator(ejemplos, max_length=200, num_return_sequences=1, temperature=0.0, early_stopping=True, num_beams=2)
print("Respuesta del modelo (Few-shot):", respuesta[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Respuesta del modelo (Few-shot): Pregunta: ¿Cuál es la capital de Francia? Respuesta: París.
Pregunta: ¿Cuál es la capital de Alemania? Respuesta: Berlín.
Pregunta: ¿Cuál es la capital de Italia? Respuesta: Roma.
Pregunta: ¿Cuál es la capital de España? Respuesta: Madrid.


En este primer ejemplo mostrado, vemos que el modelo vuelve a otorgar una respuesta correcta, viendo que ahora contesta tal y como aparece en los ejemplos anteriores (responde la capital en la misma linea que la pregunta y añadiendo un punto al final de la respuesta).

Vamos a probar con otro ejemplo, cambiando el formado de la salida esperada, para comprobar si el modelo continúa ajustándose a la forma de respuesta esperada.

In [7]:
# Few-shot learning: Proporcionamos algunos ejemplos
ejemplos = (
    "Pregunta: ¿Cuál es la capital de Francia? Respuesta: París es la capital de Francia.\n"
    "Pregunta: ¿Cuál es la capital de Alemania? Respuesta: Berlín es la capital de Alemania.\n"
    "Pregunta: ¿Cuál es la capital de Italia? Respuesta: Roma es la capital de Italia.\n"
    "Pregunta: ¿Cuál es la capital de España? Respuesta:"
)
respuesta = generator(ejemplos, max_length=200, num_return_sequences=1, temperature=0.0, early_stopping=True, num_beams=2)
print("Respuesta del modelo (Few-shot):", respuesta[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Respuesta del modelo (Few-shot): Pregunta: ¿Cuál es la capital de Francia? Respuesta: París es la capital de Francia.
Pregunta: ¿Cuál es la capital de Alemania? Respuesta: Berlín es la capital de Alemania.
Pregunta: ¿Cuál es la capital de Italia? Respuesta: Roma es la capital de Italia.
Pregunta: ¿Cuál es la capital de España? Respuesta: Madrid es la capital de España.


Vemos como en este caso también, el modelo responde correctamente y además siguiendo la estructura de las preguntas y respuestas proporcionadas (respodiendo con "X es la capital de Y", siendo X el nombre de la capital e Y el nombre del país).

## Paso 5: Chain of Thought (Cadena de Razonamiento)

En este enfoque, enseñamos al modelo a razonar paso a paso antes de dar su respuesta final. Esto puede ser útil para mejorar la precisión de las respuestas, especialmente en tareas complejas.

En este caso, probaremos a preguntarle al modelo que nos diga si sumando una serie de cifras obtenemos un número que tenga más de una serie de cifras.

In [8]:
# Chain of Thought: Razonamiento paso a paso
razonamiento2 = (
    "Sumando los siguientes números obtienes un resultado de 3 cifras: 4, 2, 6, 23, 40\n"
    "Respuesta: Falso, la suma de 4 + 2 + 6 + 23 + 40 = 75, de dos cifras\n\n"
    "Sumando los siguientes números obtienes un resultado de 3 cifras: 20, 30, 40, 55\n"
    "Respuesta: Verdadero, la suma de 20 + 30 + 40 + 55 = 145, de 3 cifras\n\n"
    "Sumando los siguientes números obtienes un resultado de 3 cifras: 1, 4, 70\n"
    "Respuesta:"
)
respuesta = generator(razonamiento2, max_length=200, num_return_sequences=1, temperature=0.0, early_stopping=True, num_beams=2)
print("Respuesta del modelo (Cadena de razonamiento):", respuesta[0]['generated_text'])

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Respuesta del modelo (Cadena de razonamiento): Sumando los siguientes números obtienes un resultado de 3 cifras: 4, 2, 6, 23, 40
Respuesta: Falso, la suma de 4 + 2 + 6 + 23 + 40 = 75, de dos cifras

Sumando los siguientes números obtienes un resultado de 3 cifras: 20, 30, 40, 55
Respuesta: Verdadero, la suma de 20 + 30 + 40 + 55 = 145, de 3 cifras

Sumando los siguientes números obtienes un resultado de 3 cifras: 1, 4, 70
Respuesta: Falso, la suma de 1 + 4 + 70 = 74, de dos cifras

Sumando los siguientes números


Vemos que en este caso, el modelo responde correctamente. La suma de 1 + 4 + 70 es 75, número de dos cifras. Por tanto, no tenemos un número de 3 cifras, y la afirmación es falsa.

Hay disponibles corpus específicos para esta tarea, para mejorar la capacidad de comprensión de los modelos y su lógica matemática, para así lograr mejores resultados.

## Conclusión
En esta cuaderno hemos explorado las técnicas de *zero-shot*, *few-shot* y *chain of thought* aplicadas a modelos de lenguaje. Estas metodologías te permitirán saber como realizar mejores preguntas a los modelos (como crear buenos prompts), aunque estas técnicas también son usadas para instruir a los modelos en tareas específicas (recuerda que en este cuaderno hemos utilizado un modelo instruido, y no su versión base).

Si quieres ahondar más en estas técnicas, puedes probar a utilizar otros modelos y/o ejemplos.